# Tracing Basics

### Setup

Make sure you set your environment variables, including your OpenAI API key.

In [1]:
from dotenv import load_dotenv
load_dotenv()  

import os
print("LangSmith Key Set:", os.getenv("LANGCHAIN_API_KEY") is not None)


LangSmith Key Set: True


### Tracing with @traceable

The @traceable decorator is a simple way to log traces from the LangSmith Python SDK. Simply decorate any function with @traceable.

The decorator works by creating a run tree for you each time the function is called and inserting it within the current trace. The function inputs, name, and other information is then streamed to LangSmith. If the function raises an error or if it returns a response, that information is also added to the tree, and updates are patched to LangSmith so you can detect and diagnose sources of errors. This is all done on a background thread to avoid blocking your app's execution.

In [8]:
# ✅ Import necessary modules
from langchain_groq import ChatGroq
from langchain_core.messages import AIMessage, HumanMessage, SystemMessage
from typing import List
import nest_asyncio
from utils import get_vector_db_retriever

# ✅ Configuration
MODEL_PROVIDER = "groq"
MODEL_NAME = "llama-3.3-70b-versatile"
APP_VERSION = 1.0
RAG_SYSTEM_PROMPT = """You are an assistant for question-answering tasks. 
Use the following pieces of retrieved context to answer the latest question in the conversation. 
If you don't know the answer, just say that you don't know. 
Use three sentences maximum and keep the answer concise.
"""

llm = ChatGroq(model=MODEL_NAME)

nest_asyncio.apply()

retriever = get_vector_db_retriever()

def retrieve_documents(question: str):
    return retriever.invoke(question)

def generate_response(question: str, documents):
    formatted_docs = "\n\n".join(doc.page_content for doc in documents)
    messages = [
        SystemMessage(content=RAG_SYSTEM_PROMPT),
        HumanMessage(content=f"Context: {formatted_docs} \n\n Question: {question}")
    ]
    return call_groq(messages)

def call_groq(messages: List, temperature: float = 0.0):
    return llm.invoke(messages)

def langsmith_rag(question: str):
    documents = retrieve_documents(question)
    response = generate_response(question, documents)
    return response.content  

@traceable handles the RunTree lifecycle for you!

In [19]:
question = "What is prompt engineering and how is it useful?"
ai_answer = langsmith_rag(question)
print(ai_answer)

Prompt engineering is the process of designing and optimizing prompts to guide the behavior of language models, allowing users to change the way the model responds without modifying its underlying capabilities. It is useful because it provides a simple and high-ROI way to influence the model's behavior, and is often more accessible than other methods like fine-tuning. By crafting effective prompts, users can shape the model's output and achieve their desired results.


##### Let's take a look in LangSmith!

### Adding Metadata

LangSmith supports sending arbitrary metadata along with traces.

Metadata is a collection of key-value pairs that can be attached to runs. Metadata can be used to store additional information about a run, such as the version of the application that generated the run, the environment in which the run was generated, or any other information that you want to associate with a run. Similar to tags, you can use metadata to filter runs in the LangSmith UI, and can be used to group runs together for analysis.

In [14]:
from langsmith import traceable
from typing import List
from langchain_groq import ChatGroq
from langchain_core.messages import SystemMessage, HumanMessage

# Constants
MODEL_NAME = "llama-3.3-70b-versatile"
MODEL_PROVIDER = "groq"
RAG_SYSTEM_PROMPT = """You are an assistant for question-answering tasks. 
Use the following pieces of retrieved context to answer the latest question in the conversation. 
If you don't know the answer, just say that you don't know. 
Use three sentences maximum and keep the answer concise.
"""

# Initialize Groq client
llm = ChatGroq(model=MODEL_NAME)
retriever = get_vector_db_retriever()  # assumes you've already defined this

@traceable(
    metadata={"vectordb": "sklearn"}
)
def retrieve_documents(question: str):
    return retriever.invoke(question)

@traceable
def generate_response(question: str, documents):
    formatted_docs = "\n\n".join(doc.page_content for doc in documents)
    messages = [
        SystemMessage(content=RAG_SYSTEM_PROMPT),
        HumanMessage(content=f"Context: {formatted_docs} \n\n Question: {question}")
    ]
    return call_groq(messages)

@traceable(
    metadata={"model_name": MODEL_NAME, "model_provider": MODEL_PROVIDER}
)
def call_groq(
    messages: List, temperature: float = 0.0
) -> str:
    return llm.invoke(messages)

@traceable
def langsmith_rag(question: str):
    documents = retrieve_documents(question)
    response = generate_response(question, documents)
    return response.content


In [15]:
question = "What is metadata?"
ai_answer = langsmith_rag(question)
print(ai_answer)

Metadata is a dictionary of key-value pairs that can be used to store additional information about a trace, such as the environment in which it was executed or the user who initiated it. It is useful for associating extra information with a trace, and can be added via the UI or SDK.


You can also add metadata at runtime!

In [17]:
question = "How does electrical engineering work?"
ai_answer = langsmith_rag(question, langsmith_extra={"metadata": {"runtime_metadata": "foo"}})
print(ai_answer)

I don't know how electrical engineering works based on the provided context. The context appears to be related to LangSmith, prompt engineering, and large language models, but does not mention electrical engineering.


##### Let's take a look in LangSmith!